#Build BioMistral Medical RAG Chatbot using BioMistral Open Source LLM

Installation

In [3]:
!pip install langchain sentence-transformers chromadb llama-cpp-python langchain_community pypdf

ERROR: Could not find a version that satisfies the requirement langchain (from versions: none)
ERROR: No matching distribution found for langchain


importing libraries

In [4]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain

ModuleNotFoundError: No module named 'langchain_community'

Import the document


In [ ]:
loader = PyPDFDirectoryLoader("Medicalchatbot")
docs = loader.load()

In [ ]:
len(docs)

chunking


In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks=text_splitter.split_documents(docs)

In [ ]:
len(chunks)

Embeeding Creations

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = "#"

In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")

#vector store creation


In [ ]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [ ]:
query = "what is the risk of heart diseases ?"

search_results = vectorstore.similarity_search(query)

In [ ]:
search_results

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k":1})

In [ ]:
retriever.get_relevant_documents(query)

#LLM Model Loading

In [ ]:
llm = LlamaCpp(
    model_path="BioMistral-7B.Q3_K_S.gguf",
    temperature=0.2,
    max_tokens = 2048,
    top_p=1
)

Use llm and retriver and query , to generate final response

In [ ]:
template = """
<|context|>
You are an Medical Assistant that follows the instructions and generate the accurate response based on the query and the context provided.
please be truthful and give direct answers.
</s>
<|user|>
{query}
</s>
<|assistant|>
"""


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
rag_chain =(
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
response = rag_chain.invoke(query)
print(response)

In [ ]:
import sys
while True:
  user_input = input(f"Input query: ")
  if user_input =='exit':
    print("Existing...")
    sys.exit()
  if user_input=="":
    continue
  result = rag_chain.invoke(user_input)
  print("Answer :",result)